<a href="https://colab.research.google.com/github/javier-jaime/Route-Recommender/blob/main/Distance_Matrix_Neo4j_MST_Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Distance Matrix

In [ ]:
# Import necessary Libraries

!pip3 install neo4j-driver
from neo4j import GraphDatabase, basic_auth
import pandas as pd
from google.colab import drive
from google.colab import files


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Mount Google Drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Read Excel file into dataframe
cities = pd.read_csv('drive/MyDrive/Colab_Data/Western_cities.csv', index_col=0)
print(len(cities))
cities

31


,Province,City,Latitude,Longitude
13,BC,Abbotsford,49.052116,-122.329479
0,AB,Acheson,53.545914,-113.771576
1,AB,Airdrie,51.285970,-114.010620
2,AB,Balzac,51.215611,-114.004558
30,ON,Bolton,43.879548,-79.738260
14,BC,Burnaby,49.243380,-122.972545
3,AB,Calgary,51.046095,-114.065465
15,BC,Delta,49.084626,-123.057938
4,AB,Edmonton,53.546205,-113.491241
16,BC,Errington,49.289463,-124.370630


 From OSRM (Open Source Routing Machine)
 https://github.com/Project-OSRM/osrm-backend/wiki/Server-api

In [ ]:
# Test the API with the coordinates from Calgary and Edmonton
route_uri = 'http://router.project-osrm.org/route/v1/driving/'
loc_pair = '-114.065465,51.046095;-113.507996,53.535411'
result = pd.read_json( route_uri + loc_pair,lines=True)
distance = result['routes'][0][0]['distance']
duration = result['routes'][0][0]['duration']
print("The distance is",distance/1000,"Kms and the duration is", round(duration/60,2),"minutes")

The distance is 297.675 Kms and the duration is 214.2 minutes


In [ ]:
# Create a new dataframe for the Distance Matrix
matrix = pd.DataFrame()
origin = []
destination = []
durations = []
distances = []

# Loop trough cites to fill the Distance Matrix
for i in range(len(cities)):
  for j in range(i+1, len(cities)):
    origin.append(cities['City'][i])
    LatO = cities['Latitude'][i]
    LonO = cities['Longitude'][i]
    destination.append(cities['City'][j])
    LatD = cities['Latitude'][j]
    LonD = cities['Longitude'][j]
    
    # Location pair of coordinates
    loc_pair = str(LonO) + ',' + str(LatO) + ';' + str(LonD) + ',' + str(LatD)

    #pass the location pairs to the OSRM API, read to json and extract routs and duration
    result = pd.read_json( route_uri + loc_pair,lines=True)
    distance = result['routes'][0][0]['distance']/1000
    duration = result['routes'][0][0]['duration']/60

    #Print results as they are coming
    print('From ',cities['City'][i],' to ',cities['City'][j],' is ',round(distance,2), 'Kms')
    
    #append result to list
    distances.append(distance)
    durations.append(duration)

matrix['Origin'] = origin
matrix['Destination'] = destination
matrix['Distance'] = distances
matrix['Duration'] = durations

From  Acheson  to  Airdrie  is  275.38 Kms
From  Acheson  to  Balzac  is  283.44 Kms
From  Acheson  to  Calgary  is  306.36 Kms
From  Acheson  to  Edmonton  is  19.98 Kms
From  Acheson  to  Grande Prairie  is  440.81 Kms
From  Acheson  to  Lethbridge  is  510.79 Kms
From  Acheson  to  Medicine Hat  is  567.86 Kms
From  Acheson  to  Okotoks  is  344.09 Kms
From  Acheson  to  Red Deer County  is  182.27 Kms
From  Acheson  to  Rocky View County  is  266.29 Kms
From  Acheson  to  Sherwood Park  is  39.64 Kms
From  Acheson  to  St. Albert  is  19.9 Kms
From  Acheson  to  Abbotsford  is  1076.52 Kms
From  Acheson  to  Burnaby  is  1129.44 Kms
From  Acheson  to  Delta  is  1135.2 Kms
From  Acheson  to  Errington  is  1282.23 Kms
From  Acheson  to  Kamloops  is  787.11 Kms
From  Acheson  to  Kelowna  is  898.99 Kms
From  Acheson  to  Ladysmith  is  1219.09 Kms
From  Acheson  to  Langley  is  1103.8 Kms
From  Acheson  to  Nanaimo  is  1240.98 Kms
From  Acheson  to  New Westminster  is  1125.25 

In [ ]:
matrix.head()

,Origin,Destination,Distance,Duration
0,Acheson,Airdrie,275.3799,200.381667
1,Acheson,Balzac,283.4424,204.070000
2,Acheson,Calgary,306.3613,221.810000
3,Acheson,Edmonton,19.9815,26.041667
4,Acheson,Grande Prairie,440.8077,311.815000


In [ ]:
# Export Distance Matrix to CSV File
matrix.to_csv('Western_Distances.csv')
!cp Western_Distances.csv "/content/drive/My Drive/Colab_Data/"

## Neo4j Driver

In [ ]:
# Connect to neo4j

uri = "bolt://3.94.203.88:7687"
driver = GraphDatabase.driver(uri, auth=basic_auth("neo4j", "bandages-hints-squeak"))
driver.verify_connectivity()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ExperimentalWarning: The configuration may change in the future.
  """


'Neo4j/4.4.10'

In [ ]:
# Delete Previous Graph

delete_graph = ''' 
MATCH (n)
DETACH DELETE n
'''

with driver.session(database="neo4j") as session:
   session.run(delete_graph)

In [ ]:
# Load Distance Matrix CSV File

load_distance = ''' 
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/javier-jaime/Route-Recommender/main/Western_Distances.csv' AS row
MERGE (o:City {name: row.Origin})
MERGE (d:City {name: row.Destination})
MERGE (o)-[r:ROAD {distance: toFloat(row.Distance), duration: toFloat(row.Duration)}]->(d)
'''

with driver.session(database="neo4j") as session:
   session.run(load_distance)

In [ ]:
# Create Graph Projection
project_graph = ''' 
CALL gds.graph.project(
  'westernCities',
  'City',
  {
    ROAD: {
      type: 'ROAD',
      properties: 'duration',
      orientation: 'UNDIRECTED'
    }
  }
)
'''

with driver.session(database="neo4j") as session:
   session.run(project_graph)

In [ ]:
# Run the Minimum Weight Spanning Tree algorithm and write back results (to the graph)
mst_query = ''' 
MATCH (n:City {name: 'Bolton'})
CALL gds.alpha.spanningTree.minimum.write('westernCities', {
  startNodeId: id(n),
  relationshipWeightProperty: 'duration',
  writeProperty: 'MINST',
  weightWriteProperty: 'writeCost'
})
YIELD preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
RETURN preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount;
'''

with driver.session(database="neo4j") as session:
   session.run(mst_query)

In [ ]:
# Query and Print results
cypher_query = '''
MATCH path = (c:City {name: 'Bolton'})-[:MINST*]-()
WITH relationships(path) AS rels
UNWIND rels AS rel
WITH DISTINCT rel AS rel
RETURN startNode(rel).name AS origin, endNode(rel).name AS destination, rel.writeCost AS duration
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      limit=500).data())
  print('Minimum Spanning Tree - Western Cities - Canada\n')
  for record in results:
    print('From', record['origin'],'to', record['destination'], 'takes', round(record['duration'],1), 'minutes')

driver.close()

Minimum Spanning Tree - Western Cities - Canada

From Bolton to Winnipeg takes 1541.5 minutes
From Winnipeg to Regina takes 412.7 minutes
From Regina to Saskatoon takes 186.0 minutes
From Regina to Medicine Hat takes 328.2 minutes
From Medicine Hat to Lethbridge takes 137.8 minutes
From Lethbridge to Okotoks takes 131.8 minutes
From Okotoks to Calgary takes 37.5 minutes
From Calgary to Kelowna takes 501.5 minutes
From Kelowna to Kamloops takes 147.3 minutes
From Kamloops to Abbotsford takes 199.5 minutes
From Abbotsford to Langley takes 28.6 minutes
From Langley to Surrey takes 24.4 minutes
From Surrey to New Westminster takes 10.5 minutes
From New Westminster to Burnaby takes 12.4 minutes
From Burnaby to Vancouver takes 18.0 minutes
From Vancouver to Richmond takes 20.5 minutes
From Richmond to Delta takes 18.7 minutes
From Delta to Ladysmith takes 247.6 minutes
From Ladysmith to Nanaimo takes 23.1 minutes
From Nanaimo to Errington takes 41.1 minutes
From New Westminster to Port Coqui

In [ ]:
# Dijkstra Shortest Path
dijkstra_query = '''
MATCH (o:City {name: 'Bolton'})
MATCH (d:City {name: 'Calgary'})
WHERE NOT (o:City {name: 'Bolton'})-[:ROAD]->(d:City {name: 'Calgary'})
CALL gds.shortestPath.dijkstra.stream('westernCities', {
        sourceNode: o, 
        targetNode: d,
        relationshipWeightProperty: 'duration'
        })
YIELD nodeIds, costs
RETURN [x in gds.util.asNodes(nodeIds)| x.name] AS cities, costs AS duration
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(dijkstra_query,
                      limit=100).data())
  for record in results:
    print('Shortest Path Cities:', record['cities'])
    print('Shortest Path Durations:', record['duration'])

driver.close()

Shortest Path Cities: ['Bolton', 'Calgary']
Shortest Path Durations: [0.0, 2393.1316666666667]


In [ ]:
# Load Cities CSV File

load_cities = ''' 
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/javier-jaime/Route-Recommender/main/Western_cities.csv' AS row
MERGE (c:City {name: row.City})
MERGE (p:Province {name: row.Province})
SET c.latitude=toFloat(row.Latitude), 
    c.longitude=toFloat(row.Longitude), 
    c.point = point({latitude: toFloat(row.Latitude),
                    longitude: toFloat(row.Longitude)})
'''

with driver.session(database="neo4j") as session:
   session.run(load_cities)

In [ ]:
# Find N nearest cities by a given point
point = 'point({latitude: 51.046095, longitude: -114.065465})'
N=10 # Number of Cities

query = '''
MATCH (c:City)
WITH c, distance(c.point, $point) as distance
RETURN c.name, distance
ORDER BY distance
LIMIT $N
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(query, point=point, N=N).data())
  for record in results:
    print('Nereast City:', record['c.name'])
    
driver.close()


Nereast City: Airdrie
Nereast City: Balzac
Nereast City: Calgary
Nereast City: Edmonton
Nereast City: Grande Prairie
Nereast City: Lethbridge
Nereast City: Medicine Hat
Nereast City: Okotoks
Nereast City: Red Deer County
Nereast City: Acheson
